Load list of segment image filepaths.

In [25]:
from pathlib import Path
from glob import glob
import re

parent_dir = Path("data").resolve()
input_dir = parent_dir / "unlabeled-images"
frame_path_list = sorted(glob(str(input_dir/"**"/"*.png")))

p = re.compile(r'^.*"(.+)"_(\d+_\d+_\d+).png$')

seg_tups = [(m.group(0), m.group(1), m.group(2))  for m in
            [p.match(s) for s in frame_path_list] if m is not None]

print("Segment #: {}".format(len(seg_tups)))

Segment #: 7404


Label a batch of images.

In [27]:
from pigeon import annotate
from IPython.display import display, Image

# Process images in batches of n
n = 1000
if len(seg_tups) < n:
    end = len(seg_tups)  
else:
    end = n
start = 0  

labels = ['swift', 'non-swift', "blurry"]

annotations = annotate(
    seg_tups[start:end],
    options=labels,
    display_fn=lambda seg_tup: display(Image(seg_tup[0], width=200, height=200))
    )

HTML(value='0 examples annotated, 1001 examples left')

Output()

Annotation done.


Write a batch of images.

In [28]:
output_dir = parent_dir/"labeled-images"

for annotation in annotations:
    # Check if class label directory exists, create if it doesn't
    class_path = Path(output_dir/annotation[1])
    if not class_path.exists():
        Path.mkdir(class_path)
        
    # Move image to its respective class directory
    img_path = Path(annotation[0][0]).resolve()
    img_path.replace(class_path / img_path.name)